In [1]:
system"c 40 200"

In [2]:
complete:("PJFFFFFFFFFJFJJF";enlist ",")0:`$":complete.csv";

In [3]:
final:select from complete where WEEKDAY<5,9<HOUR,HOUR<20

In [4]:
final:select from final where 40 <= (avg;traffic_load) fby traffic_station

In [5]:
final:update traffic_load:traffic_load%100 from final

In [6]:
minMaxScale:{[l]
    minL:min l;
    maxL:max l;
    ({(x-y)%(z-y)}[;minL;maxL]')l}

In [7]:
scaledRainfall:minMaxScale final[`rainfall]
scaledTemperature:minMaxScale final[`temperature]

final[`rainfall]:scaledRainfall
final[`temperature]:scaledTemperature

In [8]:
show scaledRainfall
show scaledTemperature

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0..
0.2666667 0.2666667 0.2666667 0.2666667 0.2666667 0.2666667 0.2666667 0.2666667 0.2666667 0.2666667 0.2666667 0.2666667 0.2666667 0.2666667 0.2666667 0.2666667 0.2666667 0.2666667 0.2666667 0.26666..


In [9]:
time_window:{[tt;data;lb]
    lb:lb+1;
    op:$[tt=`train;#;_];                                      / `train or `test decide the operator
    m:`rainfall`temperature`traffic_load`HOUR`WEEKDAY;        / the 5 columns we need
    data:?[data;();`traffic_station;m!({(y;(-;(count;x);80);x)}[;op]')m]; / first 80 or until the last 80 depending on operator 
    sw:{({y#z _x}[x;5;]')til count b:y _x}[;lb];              / sliding window function. takes turbomatrix and divides into chunks of 5x5
    gl:{y _(flip x)[2]}[;lb];                                  / gets the load (y data)
    toMatrix:{({[t;i]value t[i]}[x;]')til count x:flip x};    / table to matrix
    data:(toMatrix')data;                                     / convert each subtable (data is a keyed table) to a matrix
    X:(sw')data;                                              / apply sliding window to get X
    y:(gl')data;                                              / and gl to get y
    (X;y)                                                     / return both
    }

In [10]:
train:time_window[`train;final;5]

In [11]:
train[0][3403][0]

0f 0.2242424 0.4975 10 3
0f 0.3333333 0.525  11 3
0f 0.4424242 0.4725 12 3
0f 0.5151515 0.4975 13 3
0f 0.5818182 0.58   14 3


In [12]:
last train[0][3403]

0f 0.4       0.3875 17 0
0f 0.3212121 0.515  18 0
0f 0.3030303 0.34   19 0
0f 0.3212121 0.465  10 1
0f 0.2666667 0.5125 11 1


In [13]:
train[1][3403]

0.4875 0.4725 0.425 0.475 0.5425 0.4725 0.4525 0.4675 0.58 0.4225 0.3925 0.4925 0.5625 0.5775 0.3925 0.485 0.4325 0.57 0.49 0.3325 0.37 0.4325 0.4425 0.4475 0.26 0.445 0.355 0.44 0.4775 0.33 0.1975..


In [14]:
test:time_window[`test;final;5]

In [15]:
test[0][3403][0]

0f 0.3515152 0.45   14 1
0f 0.3757576 0.4525 15 1
0f 0.369697  0.4775 16 1
0f 0.3515152 0.54   17 1
0f 0.3454545 0.5225 18 1


In [16]:
last test[0][3403]

0f 0.569697  0.4775 13 4
0f 0.5575758 0.3875 14 4
0f 0.5454545 0.42   15 4
0f 0.5454545 0.4275 16 4
0f 0.5030303 0.5025 17 4


In [17]:
test[1][3403]

0.4275 0.475 0.435 0.54 0.51 0.4725 0.4725 0.495 0.4775 0.4625 0.465 0.495 0.475 0.4525 0.5175 0.5175 0.6325 0.4925 0.5075 0.545 0.485 0.4725 0.4825 0.51 0.2825 0.35 0.4075 0.495 0.375 0.3525 0.26 ..


---

In [18]:
system"l p.q"

In [19]:
/ pip install keras tensorflow

In [26]:
/ import chosen model and required layers
model:.p.import[`keras.models;`:Sequential][]
dense:.p.import[`keras.layers]`:Dense
lstm:.p.import[`keras.layers]`:LSTM
dropout:.p.import[`keras.layers]`:Dropout

In [27]:
/ build the model

// changed input_shape from [None,5] to [5,5]

model[`:add]lstm[500;`return_sequences pykw 1b;`input_shape pykw (5,5)];
model[`:add]lstm[250;`return_sequences pykw 1b];
model[`:add]lstm[50;`return_sequences pykw 0b];
model[`:add]dropout[0.2];
model[`:add]dense[1];
model[`:compile][`loss pykw`mae;`optimizer pykw`adam];
model[`:summary][];

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 5, 500)            1012000   
                                                                 
 lstm_4 (LSTM)               (None, 5, 250)            751000    
                                                                 
 lstm_5 (LSTM)               (None, 50)                60200     
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 1,823,251
Trainable params: 1,823,251
Non-trainable params: 0
_________________________________________________________________


In [36]:
station_data:train[0][3403]

In [37]:
.p.set[`station_data;station_data]

In [43]:
np:.p.import[`numpy]

In [50]:
np_station_data:(np[`:array][.p.eval"station_data"])

In [51]:
epochs:3
batchsz:64
start:.z.T
history:model[`:fit][np_station_data;train[1][3403];`epochs pykw epochs;`batch_size pykw batchsz;`validation_split pykw 0.2;`verbose pykw 1;`shuffle pykw 0b]
-1"Training continuous model for ",string[epochs]," epochs took ",string .z.T-start;

Epoch 1/3
2/2 [==============================] - 6s 1s/step - loss: 0.5929 - val_loss: 0.4785
Epoch 2/3
2/2 [==============================] - 0s 116ms/step - loss: 0.3512 - val_loss: 0.2401
Epoch 3/3
2/2 [==============================] - 0s 117ms/step - loss: 0.2476 - val_loss: 0.2418
Training continuous model for 3 epochs took 00:00:05.986


In [52]:
result:model[`:predict][train[0][3403]]`

4/4 [==============================] - 1s 20ms/step


In [53]:
result

0.2241465
0.2266018
0.2286546
0.2304788
0.232121 
0.2336741
0.2355167
0.2375846
0.2387517
0.237515 
0.2308107
0.2329344
0.2346323
0.2361424
0.2375651
0.2389983
0.2385278
0.2362039
0.2308013
0.2212631
0.2049485
0.2085505
0.2114858
0.2139971
0.2161809
0.2181741
0.2200732
0.2212496
0.2209727
0.2182668
0.2107917
0.2141236
0.2167989
0.2191098
0.2210683
0.2228802
0.2247825
..
